## Transmitter

In [4]:
# Import functions and libraries
import numpy as np
import matplotlib.pyplot as plt
import queue as Queue
import time
import sys
import copy

from numpy import pi
from numpy import sin
from numpy import zeros
from numpy import r_
from numpy import ones
from scipy import signal
from scipy import integrate
import threading

from numpy import mean
from numpy import power
from numpy.fft import fft
from numpy.fft import fftshift
from numpy.fft import ifft
from numpy.fft import ifftshift
import bitarray
from  scipy.io.wavfile import read as wavread
import newax25 as ax25

import multiprocessing

from math import gcd
import sounddevice as sd
import RPi.GPIO as GPIO
from functools import reduce
from numpy import ones,zeros, pi, cos, exp, sign
from queue import *

import zlib
from PIL import Image
import scipy

import numpy.ctypeslib as npct
from ctypes import c_int
from ctypes import c_float

from trans_utils import *
from proc_funs import *

%matplotlib inline

NameError: name 'np' is not defined

In [2]:
sd.query_devices()

NameError: name 'sd' is not defined

In [3]:
builtin_idx = 0
usb_idx = 2
sd.default.samplerate=48000
sd.default.channels = 1

In [4]:
# def queueREPLAY_callback(indata,outdata, frames, time, status):
#     if status:
#         print(status)
#     outdata[:] = indata
#     Qin.put( indata.copy()[:,0] )  # Global queue

## Make Modem

Set up modem arguments

In [5]:
Abuffer = 512
Nchunks = 1
fs = 48000
modem = TNCaprs(fs = fs ,Abuffer = 512,Nchunks = 1)

Set up GPIO pins

In [6]:
GPIO.setmode(GPIO.BOARD)
PTT = 12
GPIO.setup(PTT, GPIO.OUT, initial = 0)

## Transmitter Setup

In [49]:
callsign = "KM6QGZ"
img_f_name = 'calBlue.tiff'
trans_scalar = 0.2
comp_val = 1
num_preflags = 80
num_postflags = 80
for_proc_fun, rev_proc_fun = no_proc()

## Process Image to Bytes and Compress

In [55]:
bytes_to_comp = for_proc_fun(img_f_name)
bytes_comp = zlib.compress(bytes_to_comp, comp_val)
print('Compressed by: ', comp_rate(bytes_to_comp, bytes_comp))

Compressed by:  0.4039560942144981


## Make and Modulate Packets

In [56]:
print("Putting packets in Queue")

# For you to complete
Qout = Queue()
dest_int = 1
block_size = 256
q_test = Queue()
for start_ind in range(0,len(bytes_comp), block_size):
    curr_block = bytes_comp[start_ind:min(start_ind+block_size, len(bytes_comp))]
    q_test.put(curr_block)
    Qout.put(modem.modulatPacketAX25('', b'', str(dest_int), curr_block, preflags=num_preflags, postflags=num_postflags))
    dest_int += 1
print("Done")

Putting packets in Queue
Done


## Transmit Packets

In [60]:
starttime = time.time()

GPIO.output(PTT, GPIO.HIGH)
time.sleep(0.25)
num_pks = Qout.qsize()
for n in range(Qout.qsize()):
    curr_packet = Qout.get()
    print('Packet '+str(n+1)+'/'+str(num_pks))
    sd.play(curr_packet*trans_scalar,device=usb_idx,blocking=True)
GPIO.output(PTT, GPIO.LOW)

print("Done")
print(time.time() - starttime)

Packet 1/21
Packet 2/21
Packet 3/21
Packet 4/21
Packet 5/21
Packet 6/21
Packet 7/21
Packet 8/21
Packet 9/21
Packet 10/21
Packet 11/21
Packet 12/21
Packet 13/21
Packet 14/21
Packet 15/21
Packet 16/21
Packet 17/21
Packet 18/21
Packet 19/21
Packet 20/21
Packet 21/21
Done
31.56394124031067
